# Experiment 4 Bipartite n-m graphs experiment

A two layer experiment that aims to benchmark barycenter, sifting, and median heuristic algorithms to n-m bipartite graphs generated using NetworkX.
In this experiment, singleton nodes are now considered to provide the correct density for smaller graphs since smaller graphs are computationally feasible in our study.


For debugging, most of the functions presented here were copied from the original source files. 

## Design 

n-m bipartite graphs with singleton consideration
- n is even (4 6 8 10)
- for every n, m is  range[n/2,n],step=1

For bipartite graphs, where the singletons are placed does not matter when it comes to solutions. 


In [ ]:
# Imports 

import sys
import os
import random
import json
import copy
import time
import pandas as pd
import numpy as np
import math
import networkx as nx 
import matplotlib.pyplot as plt
import itertools
from networkx.algorithms import bipartite
from itertools import combinations, permutations
from typing import Dict, Union, List, Set
from concurrent.futures import ProcessPoolExecutor
%load_ext cython

## Graph Utilities

### Graph generator/s

In [2]:
def forced_density_gen_bip_graph(n1, n2, density):
    """
    Generate a bipartite graph with a specified edge density.

    Args:
        n1 (int): Number of nodes in the first partition (layer 0).
        n2 (int): Number of nodes in the second partition (layer 1).
        density (float): Desired edge density (0 < density ≤ 1), defined as |E| / (|V1| * |V2|).

    Returns:
        tuple: (nodes, edges, B, top_nodes, bottom_nodes)
            - nodes: List of dictionaries with "id" and "depth".
            - edges: List of dictionaries with "nodes" as a pair of connected node IDs.
            - B: NetworkX bipartite graph.
            - top_nodes: Set of nodes in the first partition.
            - bottom_nodes: Set of nodes in the second partition.
    """

    # Initialize bipartite graph
    B = nx.Graph()
    top_nodes = set(range(1, n1 + 1))
    bottom_nodes = set(range(n1 + 1, n1 + n2 + 1))

    B.add_nodes_from(top_nodes, bipartite=0)
    B.add_nodes_from(bottom_nodes, bipartite=1)

    # Compute the exact number of edges required
    max_edges = n1 * n2
    num_edges = max(1, min(int(math.ceil(density * max_edges)), max_edges))  # Ensure valid range

    edges = set()

    # Step 1: Shuffle
    top_list = list(top_nodes)
    bottom_list = list(bottom_nodes)
    random.shuffle(top_list)
    random.shuffle(bottom_list)

    # note that katapat nya yung meron, 
    # for i in range(max(n1, n2)):
    #     u = top_list[i % n1]  # Cycle through top nodes
    #     v = bottom_list[i % n2]  # Cycle through bottom nodes
    #     edges.add((u, v))
    #     B.add_edge(u, v)

    # Step 2: Randomly add edges based on density (no forced connections)
    while len(edges) < num_edges:
        u = random.choice(top_list)
        v = random.choice(bottom_list)
        if (u, v) not in edges:
            edges.add((u, v))
            B.add_edge(u, v)

    # Convert to required format
    nodes = [{"id": f"u{node}", "depth": 0} for node in top_nodes] + \
            [{"id": f"u{node}", "depth": 1} for node in bottom_nodes]

    edges = [{"nodes": [f"u{u}", f"u{v}"]} for u, v in edges]
    
    return nodes, edges, B, top_nodes, bottom_nodes

### utilities

- parse_edges
- cross_count_optimized

In [3]:
def parse_edges(edges, top_nodes, bottom_nodes):
    """
    Parse edges from the given format and map them to integers corresponding to top and bottom nodes.
    Args:
        edges (list): List of edges in the format [{'nodes': ['u0', 'u6']}, ...].
        top_nodes (list): List of top-layer node IDs (e.g., [0, 1, 2]).
        bottom_nodes (list): List of bottom-layer node IDs (e.g., [3, 4, 5, 6, 7]).
    
    Returns:
        list: List of tuples representing edges as (top_node, bottom_node).
    """
    parsed_edges = []
    for edge in edges:
        u, v = edge['nodes']
        # Convert 'uX' to integer node IDs
        u_id = int(u[1:])  # Remove 'u' and convert to integer
        v_id = int(v[1:])
        if u_id in top_nodes and v_id in bottom_nodes:
            parsed_edges.append((u_id, v_id))
        elif v_id in top_nodes and u_id in bottom_nodes:
            parsed_edges.append((v_id, u_id))
    # print("DEBUG: parsed_edges internal", parsed_edges, "vs", edges, "nodes",top_nodes, bottom_nodes)
    return parsed_edges


In [ ]:
def binary_search_first_smaller(arr, v, lower_bound, upper_bound, index_references, v_index):
    """
    Binary search to find the rightmost index in 'arr' where the value is smaller than 'v'.
    The search starts from 'lower_bound' and ends at 'upper_bound' to optimize performance.

    Args:
        arr (list[str]): The sorted list of neighbor nodes.
        v (str): The node to compare against.
        lower_bound (int): The starting index for the search.
        upper_bound (int): The ending index for the search.
        index_references (dict): Dictionary mapping nodes to their fixed_layer indices.
        v_index (int): The index of the node 'v' in the fixed layer.

    Returns:
        int: The index of the last element smaller than 'v', or -1 if none exist.
    """
    left, right = lower_bound, upper_bound
    result = -1  # Default to -1 (not found)

    while left <= right:  # Fix condition to include rightmost element
        mid = (left + right) // 2
        # print(f"DEBUG INSIDE BINSEARCH arr[mid]: {arr[mid]}, left: {left}, right: {right}, mid: {mid}")

        if index_references[arr[mid]] < v_index:
            result = mid  # Update result, but keep searching to the right
            left = mid + 1
        else:
            right = mid - 1  # Move left to find a smaller value

    return result  # Final rightmost valid index

def cross_count_optimized(fixed_layer: list[str], free_layer: list[str], edges: list):
    crossing_total = 0
    
    fixed_layer = [f"u{node}" if isinstance(node, int) else node for node in list(fixed_layer) ]
    free_layer =  [f"u{node}" if isinstance(node, int) else node for node in list(free_layer) ]

    fixed_layer_dict = {node: index for index, node in enumerate(fixed_layer)}
    free_layer_dict = {node: index for index, node in enumerate(free_layer)}

    neighbor_dict = {node: [] for node in free_layer}
    easy_free = set(free_layer)
    easy_fixed = set(fixed_layer)

    for edge_data in edges:
        u, v = edge_data["nodes"]
        if u in easy_free and v in easy_fixed:
            neighbor_dict[u].append(v)
        elif v in easy_free and u in easy_fixed:
            neighbor_dict[v].append(u)

    # Sort neighbors based on their position in fixed_layer
    for node in neighbor_dict:
        neighbor_dict[node].sort(key=lambda x: fixed_layer_dict[x])

    #### CROSSING PROPER ####
    for i, u_node in enumerate(free_layer):
        u_neighbors = neighbor_dict[u_node]
        u_prime_nodes = free_layer[i + 1:]
        # print("")
        # print("u_node ", u_node, ";;;u_prime nodes > u_node: ",u_prime_nodes)
        for u_prime in u_prime_nodes:
          u_prime_neighbors = neighbor_dict[u_prime]
          lb = 0   # 0 indexed as opposed to pseudocode
          ub = len(u_prime_neighbors) - 1  # 0 indexed as opposed to pseudocode
          # print(f"DEBUG u-prime-neighbors: {u_prime_neighbors} of u-prime {u_prime}")
          for v in u_neighbors:
              result = binary_search_first_smaller(u_prime_neighbors, v, lb, ub, fixed_layer_dict, fixed_layer_dict[v]) ##, edit it must be based on indices not the values of the elements themselves
              if result != -1:
                crossing_total += result + 1

    return crossing_total

[comment]: <> (Most of the code are copy-pasted from their original files)